In [ ]:
import keras
import cv2
import os
from keras.applications import InceptionV3
from keras.applications import MobileNet
from keras.applications import ResNet50



conv_base = InceptionV3(weights='imagenet',
                  include_top=True,
                  input_shape=(224, 224, 3))

In [ ]:
%timeit pass

import os
import numpy as np
from keras.preprocessing.image import ImageDataGenerator

train_dir="./hand_dataset3/train"
test_dir="./hand_dataset3/test"

datagen = ImageDataGenerator(rescale=1./255)
batch_size = 25

def extract_features(directory, sample_count):
    features = np.zeros(shape=(sample_count,1000))
    labels = np.zeros(shape=(sample_count))
    generator = datagen.flow_from_directory(
        directory,
        target_size=(224, 224),
        batch_size=batch_size,
        class_mode='binary')
    i = 0
    for inputs_batch, labels_batch in generator:
        features_batch = conv_base.predict(inputs_batch)
        features[i * batch_size : (i + 1) * batch_size] = features_batch
        labels[i * batch_size : (i + 1) * batch_size] = labels_batch
        i += 1
        if i * batch_size >= sample_count:
            # Note that since generators yield data indefinitely in a loop,
            # we must `break` after every image has been seen once.
            break
    return features, labels, generator.class_indices

train_features, train_labels, dictionary = extract_features(train_dir, 350)
test_features, test_labels, dic2 = extract_features(test_dir, 200)

In [ ]:
train_features = np.reshape(train_features, (-1, 1000))
test_features = np.reshape(test_features, (-1,1000))

from sklearn.neighbors import KNeighborsClassifier
neigh = KNeighborsClassifier(n_neighbors=3)
neigh.fit(train_features, train_labels) 
print(neigh.score(test_features, test_labels))


file = open("data.pkl", "wb")
pickle.dump(neigh, file)
file.close()


file = open("data.pkl", "rb")
neigh = pickle.load(file)
file.close()